In [7]:
import os
import re
from datetime import datetime
import pandas as pd
import numpy as np

DATA_DIR = "data"
RAW_PATH = os.path.join(DATA_DIR, "raw.csv")
CLEAN_PATH = os.path.join(DATA_DIR, "clean.pkl")

os.makedirs(DATA_DIR, exist_ok=True)

if not os.path.exists(RAW_PATH):
    raise FileNotFoundError(f"No se encontró {RAW_PATH}. Ejecuta primero generate_synthetic.py")

# 1) Cargar crudo
df = pd.read_csv(RAW_PATH, dtype=str).replace({"nan": None})

# 2) Normalizaciones básicas
df.columns = [c.strip().lower() for c in df.columns]
df["name"] = df["name"].astype(str).str.strip()
df["email"] = df["email"].astype(str).str.strip().str.lower()
df["country"] = df["country"].astype(str).str.strip().str.upper().replace({"NONE": np.nan, "NAN": np.nan, "": np.nan})

# 3) user_id a int, cuando no se pueda -> NaN
df["user_id"] = pd.to_numeric(df["user_id"], errors="coerce")

# 4) amount a float (removiendo símbolos y vacíos)
def parse_amount(x):
    if pd.isna(x):
        return np.nan
    s = str(x)
    s = re.sub(r"[^0-9.,-]", "", s)
    s = s.replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return np.nan
df["amount"] = df["amount"].apply(parse_amount)

# 5) fechas a datetime (múltiples formatos)
def parse_date(x):
    for fmt in ("%Y-%m-%d", "%d-%m-%Y", "%d/%m/%Y"):
        try:
            return datetime.strptime(str(x), fmt)
        except Exception:
            continue
    return pd.NaT
df["created_at"] = df["created_at"].apply(parse_date)

# 6) validar email simple
email_re = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,}$")
df["email_valid"] = df["email"].apply(lambda e: bool(email_re.match(str(e))) if pd.notna(e) else False)

# 7) limpiar países a catálogo simple y NaN si no conocido
valid_countries = {"PE", "MX", "CO", "AR"}
df["country"] = df["country"].apply(lambda c: c if c in valid_countries else np.nan)

# 8) drop duplicados exactos por (user_id, email, created_at)
df = df.drop_duplicates(subset=["user_id", "email", "created_at"], keep="first")

# 9) reglas de calidad mínimas
df = df[(df["user_id"].notna()) & (df["email_valid"]) & (df["created_at"].notna())]

# 10) guardar como PKL
df.to_pickle(CLEAN_PATH, protocol=4)

print(f"Datos limpios guardados en {CLEAN_PATH} — filas: {len(df)}")

Datos limpios guardados en data\clean.pkl — filas: 0
